#### Creating xacro file - Robotic Arm Example and many more

Use xacro files so that we can use macros for computation.

Begin xml file with following lines

In [ ]:
<?xml version="1.0"?>
<robot name="robotic_arm" xmlns:xacro="http://www.ros.org/wiki/xacro">

Here robot description is with joints and link descriptions.

Joints form skeleton of robot where we define each 

name and type of joint            :  joint name="S_joint_x" type="continuous"

Joint between which links         :  Parent link name and child link name

joint location & orientation      :  origin xyz="0.0 0.0 0.0" rpy="0.0 0.0 0.0"

axis of action                    :  axis xyz="0.0 0.0 1.0"

limits on action                  :  limit lower="-0.05" upper="0.1" effort="1000" velocity="5"

In [ ]:
 <joint name="Index_finger_joint" type="prismatic">
        <parent link="Wrist"/>
        <child link="Index_finger"/>
        <origin xyz="-0.05 0 .1" rpy="0 0 0" />
        <axis xyz="1 0 0" />
        <limit lower="-0.05" upper="0.1" effort="1000" velocity="5"/>
</joint>

Link description form the 


colour

mass 

intertia 

collision boundaries

location of the centre of the shape and centre of collision boundaries are mentioned along with it 

In [ ]:
#link shape and size
<geometry>
    <box size="0.02 0.1 0.3"/>
</geometry>

#color & texture
<material name="Orange">
    <color rgba="${255/255} ${108/255} ${10/255} 1"/>
    <texture filename="file_path"/>
</material>

#mass and inertia
<inertial>
    <origin xyz="0 0 ${0.3/2}" rpy="0.0 0 0.0"/>  # location of centre of the shape and orientation of shape w.r.t parent                                                        joint axis of action
    <mass value="1"/>
    <inertia ixx="0.023" iyy="0.023" izz="0.023" ixy="0.0" ixz="0.0"  iyz="0.0" />
</inertial>

#collision boundaries shape
<collision>
    <origin xyz="0 0 ${0.3/2}" rpy="0.0 0 0.0"/>    # location of centre of the collison boundary shape and orientation of                                                         shape w.r.t parent joint axis of action
    <geometry>
        <box size="0.15 0.1 0.1"/>
    </geometry>
</collision>


#### To avoid confusion 

First create link names alone with out description of link.

Create skeliton of robot with joint description

Edit each link to add full description of links

Note that orientation of each shape is by default allignes with the axis of action of parent joint.

Orign in link description defines the centre of the shape.

#### To control the robot in GAZEBO

In [ ]:
#keep controller.yaml file in config folder in workspace
arm_controller:
  type: "position_controllers/JointTrajectoryController"
  joints:
    - S_joint_x
    - S_joint_y
    - E_joint
    - W_joint

# Here Joints to which controller need to be enabled will be listed and the type of controller is also mentioned.

In [ ]:
# In launch file inform the location of controller file
<rosparam file="$(find custom_robotic_arm)/config controller.yaml"/>
<node name="controller_spawner" pkg="controller_manager" type="spawner" args="arm_controller"/>

In [ ]:

# tell gazebo that we have a controller for this urdf file by adding this in urdf file
<gazebo>
    <plugin name="control"     filename="libgazebo_ros_control.so"/>
</gazebo>


we can see the list of topics in which controller topics will be published once we launch the project using launch file

rostopic list


Once launched, controller_manager will be publishing the control actions in a topic. There will be message format fixed by controller package in which we can communicate with actions. List of nodes in control actions can be seen by command



In [ ]:

rostopic list

#response will be 

/arm_controller/command
/arm_controller/follow_joint_trajectory/cancel
/arm_controller/follow_joint_trajectory/feedback
/arm_controller/follow_joint_trajectory/goal
/arm_controller/follow_joint_trajectory/result
/arm_controller/follow_joint_trajectory/status
/arm_controller/state
/clock
/gazebo/link_states
/gazebo/model_states
/gazebo/parameter_descriptions
/gazebo/parameter_updates
/gazebo/set_link_state
/gazebo/set_model_state
/rosout
/rosout_agg

where follow_joint_trajectory is an action created by controller_manager. And message protocol(type) to communicate with this action can be seen by using autofill in terminal.  

In [ ]:
#Pressing tab after header we get header format by autofill as shown below

rostopic pub /arm_controller/command trajectory_msgs/JointTrajectory "header:
  seq: 0
  stamp:
    secs: 0
    nsecs: 0
  frame_id: ''
joint_names:
- ''
points:
- positions: [0]
  velocities: [0]
  accelerations: [0]
  effort: [0]
  time_from_start: {secs: 0, nsecs: 0}" 


No using this message we can send the goal positions and velocities etc to action node for required joint name as shown below

In [ ]:
rostopic pub /arm_controller/command trajectory_msgs/JointTrajectory '{joint_names:["S_joint_x","S_joint_y","E_joint","W_joint"],points:[{positions:[-0.8,0.5,-0.75,0.2],time_from_start:[1,0]}]}'
